importing libraries

In [23]:
import pandas as pd
import numpy as np
from openai import OpenAI
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score

Reduced Dataset is the portion of the initial Dataset that was complete

In [3]:
reduced_df = pd.read_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_df.csv')
reduced_df

/tmp/ipykernel_3643/1069376804.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_df = pd.read_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_df.csv')


,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,memory,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,3108.0,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,3080.0,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,3096.0,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,2588.0,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,3080.0,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,6818156,2073,s921588169,p04050,u748871552,1470008541,C++,C++11 (GCC 4.8.1),cpp,Accepted,...,256.0,1080,NaN,1,s921588169,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"#include <bits/stdc++.h>\n\n#define REP(i,n) f...",Here is the code summary format for the given ...
499996,6818157,2074,s191865152,p04050,u955723966,1469987618,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1066,NaN,1,s191865152,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\n\nusing namespace st...,Here is the code summary format for the given ...
499997,6818166,2083,s564884250,p04050,u572012381,1469778717,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1577,NaN,1,s564884250,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here is the code summary format for the given ...
499998,6818168,2085,s162996393,p04050,u657011188,1469479168,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1021,NaN,1,s162996393,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here is the code summary format for the given ...


Using a small sample of the first 100 entries as a test for the summarisation pipeline

In [4]:
df = reduced_df.copy()
df_sample = df.head(100).reset_index(drop=True)


In [5]:
df = df_sample
df

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,memory,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,3108.0,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,3080.0,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,3096.0,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,2588.0,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,3080.0,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5407,8972,s729213997,p00000,u910106747,1424452541,C++,C++,cpp,Accepted,...,1116.0,348,1/1,1,s729213997,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"\n#include ""iostream""\n\nusing namespace std;\...",Here's the code summary for the given problem ...
96,5415,8980,s317924322,p00000,u460034312,1424877773,C++,C++,cpp,Accepted,...,1116.0,200,1/1,1,s317924322,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
97,5416,8981,s615960680,p00000,u014398719,1425114218,C++,C++,cpp,Accepted,...,1116.0,311,1/1,1,s615960680,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n\nusing namespace std;\n\...,Here's the code summary for the given problem ...
98,5538,9103,s871011653,p00000,u080308822,1425519637,C++,C++,cpp,Accepted,...,1116.0,189,1/1,1,s871011653,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...


Using the DeepSeek model to generate summaries for the code submissions in the dataset.              In this test the base deepseek model is used and gives satisfactoy results at least for the sample data ,in proper implementation the plan is to use the deepsek reasoning model for better results.

In [8]:
client = OpenAI(api_key="sk-9a18103e17be4329a06b65b1520c5d9b", base_url="https://api.deepseek.com")

def GenerateSummary(text):

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": """You are are an assistant/tutor helping users understand various code snippets by giving a general summary and as much as poosible give them  in a general summary form for specifically c and c++ code like this;Code Summary Format

    1. Function/Method Name:
    process_data

    2. Purpose:
    Briefly describe what the code is intended to do.
    E.g., Processes and cleans input data by removing null values and standardizing formats.

    3. Inputs:

    dataframe (pandas.DataFrame): The input dataset containing raw values.

    columns (List[str]): Specific columns to be processed.


    4. Outputs:

    cleaned_dataframe (pandas.DataFrame): A cleaned version of the input dataset.


    5. Key Operations/Logic:

    Drops rows with null values in specified columns

    Converts column names to lowercase

    Normalizes date formats


    6. Dependencies:

    pandas

    datetime


    7. Edge Cases Handled:

    Returns an empty DataFrame if all rows contain nulls

    Skips processing if columns is empty


    8. Example Usage (Optional):

    clean_df = process_data(df, ["date", "price", "location"])"""},
            {"role": "user", "content": f"{text}"},
        ],
        stream=False
    )
    return response



In [9]:
df = df_sample
df['GenSummary'] = df['code'].apply(lambda x: GenerateSummary(x).choices[0].message.content if pd.notnull(x) else None)
df

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here's the summary of the provided C++ code:\n...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,### Code Summary Format \n\n1. **Function/Met...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5407,8972,s729213997,p00000,u910106747,1424452541,C++,C++,cpp,Accepted,...,348,1/1,1,s729213997,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"\n#include ""iostream""\n\nusing namespace std;\...",Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...
96,5415,8980,s317924322,p00000,u460034312,1424877773,C++,C++,cpp,Accepted,...,200,1/1,1,s317924322,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...
97,5416,8981,s615960680,p00000,u014398719,1425114218,C++,C++,cpp,Accepted,...,311,1/1,1,s615960680,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n\nusing namespace std;\n\...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...
98,5538,9103,s871011653,p00000,u080308822,1425519637,C++,C++,cpp,Accepted,...,189,1/1,1,s871011653,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...


In [16]:
df.to_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_sample_df.csv')
df

#df_sample

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here's the summary of the provided C++ code:\n...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,### Code Summary Format \n\n1. **Function/Met...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5407,8972,s729213997,p00000,u910106747,1424452541,C++,C++,cpp,Accepted,...,348,1/1,1,s729213997,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"\n#include ""iostream""\n\nusing namespace std;\...",Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...
96,5415,8980,s317924322,p00000,u460034312,1424877773,C++,C++,cpp,Accepted,...,200,1/1,1,s317924322,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...
97,5416,8981,s615960680,p00000,u014398719,1425114218,C++,C++,cpp,Accepted,...,311,1/1,1,s615960680,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n\nusing namespace std;\n\...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...
98,5538,9103,s871011653,p00000,u080308822,1425519637,C++,C++,cpp,Accepted,...,189,1/1,1,s871011653,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...


for evaluating the generated sumaries BLEU and BERT score are the metrics used to compare the generated summaries with the original summaries in the dataset. the BERT score is calculated using the BERTScore library instead of a "manual" Bert model implimentation while this implimentation is faster access to the acc embedding-space is lost

In [32]:


def calculate_bleu(reference, candidate):
    reference = reference.split()
    candidate = candidate.split()
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
df['bleu_score'] = df.apply(
    lambda row: calculate_bleu(row['GenSummary'], row['summary']) if pd.notnull(row['summary']) and pd.notnull(row['GenSummary']) else None,
    axis=1
)
df['BERT_score'] = df.apply(
    lambda row: score([row["GenSummary"]], [row["summary"]], model_type="microsoft/codebert-base")[2][0].item(),
    axis=1
)
df


KeyError: 'microsoft/codebert-base'

In [33]:
df.to_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_sample_df_Evaluated.csv', index=False)
df

,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary,GenSummary,bleu_score,BERT_score
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here's the summary of the provided C++ code:\n...,0.185897,0.777436
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,### Code Summary Format \n\n1. **Function/Met...,0.034863,0.728369
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.160364,0.787236
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.051598,0.747192
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...,0.162871,0.796201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5407,8972,s729213997,p00000,u910106747,1424452541,C++,C++,cpp,Accepted,...,1,s729213997,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"\n#include ""iostream""\n\nusing namespace std;\...",Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.074141,0.741793
96,5415,8980,s317924322,p00000,u460034312,1424877773,C++,C++,cpp,Accepted,...,1,s317924322,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...,0.225518,0.815287
97,5416,8981,s615960680,p00000,u014398719,1425114218,C++,C++,cpp,Accepted,...,1,s615960680,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n\nusing namespace std;\n\...,Here's the code summary for the given problem ...,Here’s the structured summary of the provided ...,0.167847,0.789103
98,5538,9103,s871011653,p00000,u080308822,1425519637,C++,C++,cpp,Accepted,...,1,s871011653,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...,Here’s a structured summary of the provided C+...,0.079642,0.747774
